Import

In [284]:
import os
import json

import pandas as pd
from matplotlib import pyplot as plt
import tabula

DataSet

In [285]:
airports_df = pd.read_excel('assets/airports.xlsx')
flights_df = pd.read_excel('assets/flights.xlsx')
planes_df = pd.read_csv('assets/planes.csv')
airlines_df = pd.read_json('assets/airlines.json')

Preprocessing

In [286]:
# Séparation des colonnes en utilisant la virgule comme séparateur
flights_df = flights_df['year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour'].str.split(',', expand=True)

# Renommer les colonnes après la séparation
flights_df.columns = ['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight', 'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute', 'time_hour']

In [290]:
airports_head = airports_df.head()
flights_head = flights_df.head()
planes_head = planes_df.head()
airlines_head = airlines_df.head()

flights_df

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2021,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2021-01-01T05:00:00Z
1,2021,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2021-01-01T05:00:00Z
2,2021,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2021-01-01T05:00:00Z
3,2021,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2021-01-01T05:00:00Z
4,2021,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2021-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252699,2021,7,3,1059,1100,-1,1214,1210,4,MQ,3230,N522MQ,JFK,DCA,51,213,11,0,2021-07-03T11:00:00Z
252700,2021,7,3,1059,1100,-1,1207,1213,-6,US,2171,N754UW,LGA,DCA,43,214,11,0,2021-07-03T11:00:00Z
252701,2021,7,3,1101,1110,-9,1305,1240,25,9E,3910,N813AY,JFK,RIC,61,288,11,10,2021-07-03T11:00:00Z
252702,2021,7,3,1105,1055,10,1435,1358,37,B6,971,N598JB,LGA,FLL,176,1076,10,55,2021-07-03T10:00:00Z


Question 1

In [288]:
# Calcul du nombre total d'aéroports, de fuseaux horaires, et des aéroports sans passage à l'heure d'été (dst == 'N')
total_airports = len(airports_df)
airports_no_dst = len(airports_df[airports_df['dst'] == 'N'])
timezones = airports_df['tzone'].nunique()

# Calcul du nombre total de départs
total_departures = flights_df['origin'].nunique()

# Nombre de destinations
total_destinations = flights_df['dest'].nunique()

# Nombre de compagnies
total_companies = len(airlines_df)

# Nombre d'avions
total_planes = len(planes_df)

# Vérifie si les colonnes 'dep_time' et 'arr_time' ont des valeurs manquantes
flights_cancelled = flights_df[(flights_df['dep_time'] == ' ') & (flights_df['arr_time']== ' ')]

# Calcule le nombre total de vols annulés
total_cancelled_flights = len(flights_cancelled)

# Affichage des résultats
print(f"Nombre total d'aéroports : {total_airports}")
print(f"Nombre total de départs : {total_departures}")
print(f"Nombre total de destinations : {total_destinations}")
print(f"Nombre d'aéroports où on ne passe pas à l'heure d'été : {airports_no_dst}")
print(f"Nombre de fuseaux horaires différents : {timezones}")
print(f"Nombre de compagnies aériennes : {total_companies}")
print(f"Nombre d'avions : {total_planes}")
print(f"Nombre de vols annulés : {total_cancelled_flights}")

Nombre total d'aéroports : 1458
Nombre total de départs : 3
Nombre total de destinations : 103
Nombre d'aéroports où on ne passe pas à l'heure d'été : 23
Nombre de fuseaux horaires différents : 10
Nombre de compagnies aériennes : 16
Nombre d'avions : 3322
Nombre de vols annulés : 6481


Question 2

In [298]:
# Total des vols pour calculer le pourcentage
total_flights = len(flights_df)

# Calcul de l'aéroport de départ le plus emprunté
top_departure_airport = flights_df['origin'].value_counts().idxmax()
top_departure_airport_count = flights_df['origin'].value_counts().max()

# Calcul des 10 destinations les plus et les moins prisées
top_10_destinations = flights_df['dest'].value_counts().head(10)
# Calcul du pourcentage pour chaque destination
top_10_destinations_percent = (top_10_destinations / total_flights) * 100

# Jointure pour récupérer les noms complets des aéroports
top_10_destinations_named = pd.merge(
    top_10_destinations.reset_index(),  # Convertir en DataFrame
    airports_df[['faa', 'name']],  # Sélectionner les colonnes nécessaires dans airports_df
    left_on='dest',  # Faire la jointure sur 'index', qui est le code d'aéroport (dest)
    right_on='faa',   # Joindre avec la clé 'faa' dans airports_df
    how='left'        # Faire un left join pour garder toutes les destinations
)

# Ajouter la colonne du pourcentage
top_10_destinations_named['percent'] = top_10_destinations_percent.values

# Renommer les colonnes pour plus de clarté
top_10_destinations_named = top_10_destinations_named[['name', 'dest', 'percent']]

# Sélectionner et renommer les colonnes pertinentes
top_10_destinations_named = top_10_destinations_named.rename(columns={'index': 'dest'})

# Calcul des 10 destinations les moins prisées
least_10_destinations = flights_df['dest'].value_counts().tail(10)
# Jointure pour récupérer les noms complets des aéroports
least_10_destinations_named = pd.merge(
    least_10_destinations.reset_index(),  # Convertir en DataFrame
    airports_df[['faa', 'name']],  # Sélectionner les colonnes nécessaires dans airports_df
    left_on='dest',  # Faire la jointure sur 'index', qui est le code d'aéroport (dest)
    right_on='faa',   # Joindre avec la clé 'faa' dans airports_df
    how='left'        # Faire un left join pour garder toutes les destinations
)

# Renommer les colonnes pour plus de clarté
least_10_destinations_named = least_10_destinations_named[['name', 'dest']]

# Calcul des 10 avions qui ont le plus et le moins décollé
top_10_planes = flights_df['tailnum'].value_counts().head(10)
least_10_planes = flights_df['tailnum'].value_counts().tail(10)

top_10_destinations_named
# least_10_destinations_named

,name,dest,percent
0,Hartsfield Jackson Atlanta Intl,ATL,5.122990
1,Chicago Ohare Intl,ORD,5.007440
2,Los Angeles Intl,LAX,4.707088
3,General Edward Lawrence Logan Intl,BOS,4.574522
4,Orlando Intl,MCO,4.209273
5,Charlotte Douglas Intl,CLT,4.134481
6,San Francisco Intl,SFO,3.849959
7,Fort Lauderdale Hollywood Intl,FLL,3.736783
8,Miami Intl,MIA,3.536944
9,Ronald Reagan Washington Natl,DCA,2.922787
